In [1]:
import os, sys, random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img 
import torchvision.transforms as transforms

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Lambda
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [2]:
#Specificam sistemului sa nu foloseasca tot VRAM-ul pentru a evita anumite erori de sistem.
#Alocarea memoriei creste dinamic in functie de cerinte.
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# Functie definita pentru citirea datelor de antranare
def readTrainImages(path):
    train_images = []
    # Iteram prin primele 15000 de poze pentru a adauga la train_images
    for filename in sorted(os.listdir(path))[:15000]:
        # print((i,filename))
        
        # Incarcam imaginea curenta
        image = Image.open(path + "/" + filename)
        
        # Adaugam imaginea curenta la vectorul de imagini.
        train_images.append(np.array(image))
        
    # Convertim array-ul imaginilor intr-o np.array si il returnam
    return np.array(train_images)

In [4]:
# Functie definita pentru citirea datelor de validare - aceeasi functionalitate ca prima
def readValidationImages(path):
    valiadation_images = []

    for filename in sorted(os.listdir(path))[15000:17000]:
        # print((i,filename))
        image = Image.open(path + "/" + filename)
        valiadation_images.append(np.array(image))
        
    return np.array(valiadation_images)

In [5]:
# Functie definita pentru citirea datelor de testare - aceeasi functionalitate ca prima
def readTestImages(path):
    test_images = []
    
    for filename in sorted(os.listdir(path))[17000:]:
        # print((i,filename))
        image = Image.open(path + "/" + filename)
        test_images.append(np.array(image))
        
    return np.array(test_images)

In [6]:
# Functie care citeste din fisier label-urile pozelor. 
def readLabels(path,trainData=False):
    
    f = open(path)
    
    # Citim prima linie din fisier, care nu contine un label
    f.readline()
    
    labels=[]
    
    # Dintr-o linie memoram doar labelul
    for linie in f.readlines():
        labels.append(int(linie.split(',')[1]))
    f.close()
    
    return labels

In [7]:
# Ne definim cai pentru a citi datele de intrare.
pathImages = "/kaggle/input/unibuc-brain-ad/data/data"
pathLabels = "/kaggle/input/unibuc-brain-ad/data"

# pathImages = "D:/Chestii/Facultate/Anul II/Semestrul II/IA - Inteligenta Artificiala/Proiect ML/data/data"
# pathLabels = "D:/Chestii/Facultate/Anul II/Semestrul II/IA - Inteligenta Artificiala/Proiect ML/data"

In [8]:
#Citire date
train_images=readTrainImages(pathImages)
print(1)

validation_images=readValidationImages(pathImages)
print(2)

test_images=readTestImages(pathImages)
print(3)

train_labels=np.array(readLabels(pathLabels + "/train_labels.txt"))
print(4)

validation_labels=np.array(readLabels(pathLabels + "/validation_labels.txt"))
print(5)

1
2
3
4
5


In [9]:
#Definirea modelului
CNN = Sequential([
    Lambda(lambda x: x / 255.0, input_shape=(224, 224, 3)), #normalizam datele printr-un layer lambda
    Conv2D(16, (3,3), 1, activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(),
    Conv2D(32, (3,3), 1, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3,3), 1, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid') #avem un singur rezultat, anume probabilitatea ca imaginea curenta sa apartina clasei 
                                   #1 sau 0. Folosim functia de activare sigmoid tocmai pentru ca trebuie sa ne decidem 
                                   #intre 2 clase. Daca trebuia sa decidem intre mai multe clase am fi folosit softmax.
                                   #Functia sigmoid transforma input-ul in o valoare [0,1], adica o probabilitate.
])

In [10]:
# Compilam  programul
CNN.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
CNN.summary()

# Pentru debug - putem vizualiza procesul de antranre al modelului pas cu
steps='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=steps)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        1

In [11]:
# Antrenarea modelului
hist = CNN.fit(train_images,train_labels,validation_data=(validation_images,validation_labels),epochs=20,callbacks=[tensorboard_callback])

Epoch 1/20
469/469 [==============================] - 29s 43ms/step - loss: 0.3579 - accuracy: 0.8494 - val_loss: 0.3157 - val_accuracy: 0.8620
Epoch 2/20
469/469 [==============================] - 18s 38ms/step - loss: 0.3224 - accuracy: 0.8520 - val_loss: 0.3174 - val_accuracy: 0.8775
Epoch 3/20
469/469 [==============================] - 18s 39ms/step - loss: 0.3058 - accuracy: 0.8653 - val_loss: 0.2983 - val_accuracy: 0.8820
Epoch 4/20
469/469 [==============================] - 19s 40ms/step - loss: 0.2898 - accuracy: 0.8724 - val_loss: 0.3006 - val_accuracy: 0.8690
Epoch 5/20
469/469 [==============================] - 19s 40ms/step - loss: 0.2654 - accuracy: 0.8856 - val_loss: 0.3274 - val_accuracy: 0.8665
Epoch 6/20
469/469 [==============================] - 19s 40ms/step - loss: 0.2310 - accuracy: 0.9001 - val_loss: 0.3751 - val_accuracy: 0.8610
Epoch 7/20
469/469 [==============================] - 18s 39ms/step - loss: 0.1830 - accuracy: 0.9269 - val_loss: 0.4249 - val_accuracy:

In [12]:
test_labels = CNN.predict(test_images)

# Generam CSV-ul care contine label-urile la care s-a dat predict.
stringLabels = []

# Formam lista de stringuri care se va scrie in CSV.

i=17001
for lab in test_labels:
    if lab > 0.5:
        label='0' + str(i) + ',' + str(1)
    else:
        label='0' + str(i) + ',' + str(0)
    stringLabels.append(label)
    i+=1
    
np.savetxt('rezultat.csv', stringLabels, fmt='%s', header="id,class", comments='')


161/161 [==============================] - 2s 13ms/step
